In [1]:
# imports

import gym
import random as rd
from collections import deque
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
# implementing DQN class

class DQN:

    REPLAY_MEMORY_SIZE = 5000 			# number of tuples in experience replay  
    EPSILON = 1 						# epsilon of epsilon-greedy exploation
    EPSILON_DECAY = 0.999 				# exponential decay multiplier for epsilon
    HIDDEN1_SIZE = 16 					# size of hidden layer 1
    HIDDEN2_SIZE = 16 					# size of hidden layer 2
    EPISODES_NUM = 2000 				# number of episodes to train on. Ideally shouldn't take longer than 2000
    MAX_STEPS = 200 					# maximum number of steps in an episode 
    LEARNING_RATE = 0.001 				# learning rate and other parameters for SGD/RMSProp/Adam
    MINIBATCH_SIZE = 10 				# size of minibatch sampled from the experience replay
    DISCOUNT_FACTOR = 0.9 				# MDP's gamma
    TARGET_UPDATE_FREQ = 50 			# number of steps (not episodes) after which to update the target networks 
    LOG_DIR = './logs' 					# directory wherein logging takes place
    EPSILON_MIN = 0.05


    # Create and initialize the environment
    def __init__(self, env):
        self.env = gym.make(env)
        assert len(self.env.observation_space.shape) == 1
        self.input_size = self.env.observation_space.shape[0]		# In case of cartpole, 4 state features
        self.output_size = self.env.action_space.n					# In case of cartpole, 2 actions (right/left)
        self.epsilon = self.EPSILON

    # Create the Q-network
    def initialize_network(self):

        ############################################################
        # Design your q-network here.
        # 
        # Add hidden layers and the output layer. For instance:
        # 
        # with tf.name_scope('output'):
        #	W_n = tf.Variable(
        # 			 tf.truncated_normal([self.HIDDEN_n-1_SIZE, self.output_size], 
        # 			 stddev=0.01), name='W_n')
        # 	b_n = tf.Variable(tf.zeros(self.output_size), name='b_n')
        # 	self.Q = tf.matmul(h_n-1, W_n) + b_n
        #
        #############################################################
        
        # Model designed using keras layers
        self.model = keras.Sequential([
                layers.InputLayer(input_shape=(self.input_size,)),
                layers.Dense(self.HIDDEN1_SIZE, activation='relu', name='hidden1', kernel_initializer='RandomNormal'),
                layers.Dense(self.HIDDEN2_SIZE, activation='relu', name='hidden2', kernel_initializer='RandomNormal'),
                layers.Dense(self.output_size, activation='linear', name='output', kernel_initializer='RandomNormal')
        ])

        ############################################################
        # Next, compute the loss.
        #
        # First, compute the q-values. Note that you need to calculate these
        # for the actions in the (s,a,s',r) tuples from the experience replay's minibatch
        #
        # Next, compute the l2 loss between these estimated q-values and 
        # the target (which is computed using the frozen target network)
        #
        ############################################################
        
        ############################################################
        # Finally, choose a gradient descent algorithm : SGD/RMSProp/Adam. 
        #
        # For instance:
        # optimizer = tf.train.GradientDescentOptimizer(self.LEARNING_RATE)
        # global_step = tf.Variable(0, name='global_step', trainable=False)
        # self.train_op = optimizer.minimize(self.loss, global_step=global_step)
        #
        ############################################################

        # Assigned descent algo. and loss function in one line
        self.model.compile(loss=keras.losses.MeanSquaredError(),optimizer=keras.optimizers.Adam(lr=self.LEARNING_RATE))
        self.model.summary()
        
        # create a target model a clone to our model i.e. target network
        self.target_model = keras.models.clone_model(self.model)
        self.target_model.build((None, self.input_size))
        self.target_model.compile(loss=keras.losses.MeanSquaredError(),optimizer=keras.optimizers.Adam(lr=self.LEARNING_RATE))
        self.target_model.set_weights(self.model.get_weights())

        ############################################################

    def train(self, episodes_num=EPISODES_NUM):

        # Initialize summary for TensorBoard 
        summary_writer = tf.summary.create_file_writer(self.LOG_DIR)
        summary = tf.summary
        # Alternatively, you could use animated real-time plots from matplotlib 
        # (https://stackoverflow.com/a/24228275/3284912)

#         # Initialize the TF session
#         self.session = tf.Session()
#         self.session.run(tf.global_variables_initializer())

        ############################################################
        # Initialize other variables (like the replay memory)
        ############################################################
        
        # Using deque
        self.replay_buffer = deque(maxlen=self.REPLAY_MEMORY_SIZE)
        total_steps = 0

        ############################################################
        # Main training loop
        # 
        # In each episode, 
        #	pick the action for the given state, 
        #	perform a 'step' in the environment to get the reward and next state,
        #	update the replay buffer,
        #	sample a random minibatch from the replay buffer,
        # 	perform Q-learning,
        #	update the target network, if required.
        #
        #
        #
        # You'll need to write code in various places in the following skeleton
        #
        ############################################################

        for episode in range(episodes_num):

            state = np.array([self.env.reset()])

            ############################################################
            # Episode-specific initializations go here.
            ############################################################
            
            episode_length = 0
            score = 0
            
            ############################################################

            while True:
                ############################################################
                # Pick the next action using epsilon greedy and execute it
                ############################################################
                
                episode_length += 1
                total_steps += 1
                if(rd.random() < self.epsilon):
                    act = self.env.action_space.sample()
                else:
                    act = np.argmax(self.model.predict(state)[0])

                ############################################################
                # Step in the environment. Something like: 
                # next_state, reward, done, _ = self.env.step(action)
                ############################################################

                next_state, reward, done, _ = self.env.step(act)
                next_state = np.array([next_state])
                score += reward
                
                ############################################################
                # Update the (limited) replay buffer. 
                #
                # Note : when the replay buffer is full, you'll need to 
                # remove an entry to accommodate a new one.
                ############################################################

                # The max length in deque removes oldest if buffer size exceeds it
                self.replay_buffer.append((state,act,reward,next_state,done))
                state = next_state

                ############################################################
                # Sample a random minibatch and perform Q-learning (fetch max Q at s') 
                #
                # Remember, the target (r + gamma * max Q) is computed    
                # with the help of the target network.
                # Compute this target and pass it to the network for computing 
                # and minimizing the loss with the current estimates
                #
                ############################################################
                
                # not starting network update until it has a batch size elements
                if len(self.replay_buffer) == self.REPLAY_MEMORY_SIZE:
                    batch_states = np.zeros((self.MINIBATCH_SIZE,))
                    batch_targets = np.zeros((self.MINIBATCH_SIZE,))
                    replay_batch = rd.sample(self.replay_buffer,self.MINIBATCH_SIZE)
                    b = 0
                    for st, act, rwd, nst, d in replay_batch:
                        if d:
                            y = rwd
                        else:
                            y = (rwd + self.DISCOUNT_FACTOR * np.max(self.target_model.predict(nst)[0]))
                        tgt = self.model.predict(state)
                        tgt[0][act] = y
                        batch_states.append(st)
                        batch_targets.append(tgt)
                        
                    self.model.fit(batch_states, batch_targets, epochs=1, verbose = 0)
                    
                    if self.epsilon > self.EPSILON_MIN:
                        self.epsilon *= self.EPSILON_DECAY

                ############################################################
                # Update target weights. 
                #
                # Something along the lines of:
                # if total_steps % self.TARGET_UPDATE_FREQ == 0:
                # 	target_weights = self.session.run(self.weights)
                ############################################################

                if total_steps%self.TARGET_UPDATE_FREQ == 0:
                    self.target_model.set_weights(self.model.get_weights())

                ############################################################
                # Break out of the loop if the episode ends
                #
                # Something like:
                # if done or (episode_length == self.MAX_STEPS):
                # 	break
                #
                ############################################################
                
                if done or episode_length == self.MAX_STEPS:
                    break


            ############################################################
            # Logging. 
            #
            # Very important. This is what gives an idea of how good the current
            # experiment is, and if one should terminate and re-run with new parameters
            # The earlier you learn how to read and visualize experiment logs quickly,
            # the faster you'll be able to prototype and learn.
            #
            # Use any debugging information you think you need.
            # For instance :

            print("Training: Episode = %d, Length = %d, Global step = %d" % (episode, episode_length, total_steps))
            with summary_writer.as_default():
                summary.scalar("episode length",episode ,step=episode_length)


    # Simple function to visually 'test' a policy
    def playPolicy(self):

        done = False
        steps = 0
        state = self.env.reset()

        # we assume the CartPole task to be solved if the pole remains upright for 200 steps
        while not done and steps < 200: 
            self.env.render()
            action = np.argmax(self.target_model.predict(np.array([state]))[0])
            state, _, done, _ = self.env.step(action)
            steps += 1

        return steps

In [5]:
# Create and initialize the model
dqn = DQN('CartPole-v0')
dqn.initialize_network()

print("\nStarting training...\n")
dqn.train()
print("\nFinished training...\nCheck out some demonstrations\n")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden1 (Dense)              (None, 16)                80        
_________________________________________________________________
hidden2 (Dense)              (None, 16)                272       
_________________________________________________________________
output (Dense)               (None, 2)                 34        
Total params: 386
Trainable params: 386
Non-trainable params: 0
_________________________________________________________________

Starting training...

Training: Episode = 0, Length = 38, Global step = 38
Training: Episode = 1, Length = 66, Global step = 104
Training: Episode = 2, Length = 10, Global step = 114
Training: Episode = 3, Length = 35, Global step = 149
Training: Episode = 4, Length = 22, Global step = 171
Training: Episode = 5, Length = 36, Global step = 207
Training: Episode = 6, Length = 23, Glob

Training: Episode = 165, Length = 12, Global step = 3725
Training: Episode = 166, Length = 89, Global step = 3814
Training: Episode = 167, Length = 17, Global step = 3831
Training: Episode = 168, Length = 18, Global step = 3849
Training: Episode = 169, Length = 15, Global step = 3864
Training: Episode = 170, Length = 42, Global step = 3906
Training: Episode = 171, Length = 31, Global step = 3937
Training: Episode = 172, Length = 19, Global step = 3956
Training: Episode = 173, Length = 13, Global step = 3969
Training: Episode = 174, Length = 13, Global step = 3982
Training: Episode = 175, Length = 12, Global step = 3994
Training: Episode = 176, Length = 13, Global step = 4007
Training: Episode = 177, Length = 22, Global step = 4029
Training: Episode = 178, Length = 22, Global step = 4051
Training: Episode = 179, Length = 12, Global step = 4063
Training: Episode = 180, Length = 34, Global step = 4097
Training: Episode = 181, Length = 14, Global step = 4111
Training: Episode = 182, Length

ValueError: in user code:

    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:531 train_step  **
        y_pred = self(x, training=True)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:886 __call__
        self.name)
    C:\Progra

In [ ]:
# Visualize the learned behaviour for a few episodes
results = []
for i in range(50):
    episode_length = dqn.playPolicy()
    print("Test steps = ", episode_length)
    results.append(episode_length)
print("Mean steps = ", sum(results) / len(results))	

print("\nFinished.")
print("\nCiao, and hasta la vista...\n")

In [7]:
b = np.array([])
b.append([1,2])

AttributeError: 'numpy.ndarray' object has no attribute 'append'